In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Import all Packages**

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import backend as K
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import pyplot
from statistics import mean
import glob
import os
import cv2
import sys

# **Load the Dataset**

In [ ]:
data = pd.read_csv("../input/letterrecognition-using-svm/letter-recognition.csv",header=0)# header 0 means the first row is name of the coloumn 

 
# View sample data
data.head(10) 

In [ ]:
data['label'] = pd.factorize(data.letter)[0]

In [ ]:
data.head()

In [ ]:
del data["letter"]

In [ ]:
data.head()

In [ ]:
##########################################################
#SHARE TO TEST AND TRAIN DATA
##########################################################
x = data.iloc[:, :16]
y = data['label'].tolist()
print(x)

# Select 4000 rows data as a testing dataset
x_test = x.iloc[0:3000, :].values.astype('float32') # all pixel values 
y_test = y[0:3000] # Select label for testing data
x_train = x.iloc[3000:, :].values.astype('float32') # all pixel values 
y_train = y[3000:]

# # Share test and train data
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
y_train

# **Reshape Dataset to Have a Single Channel**

In [ ]:
x_train = x_train.reshape((x_train.shape[0], 4, 4, 1))
x_test = x_test.reshape((x_test.shape[0], 4, 4, 1))



In [ ]:
x_train.shape

# **Normalize Input**

In [ ]:
#x_train2 = x_train/255
#x_test2 = x_test/255
x_train2 = x_train/255 - 0.5
x_test2 = x_test/255 - 0.5

# Convert class labels to one-hot encoded
y_train2 = keras.utils.to_categorical(y_train)
y_test2 = keras.utils.to_categorical(y_test)

# Print labels
y_train2

# **Import Necessary CNN Building Blocks**

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from keras.layers.advanced_activations import LeakyReLU

# **Define/Create CNN Architecture**

In [ ]:
# Create a model function
def make_model():
    model = Sequential()

    model.add(Conv2D(filters = 16, kernel_size = (3, 3), padding='same', input_shape=(4, 4, 1)))
    model.add(LeakyReLU(0.1))
    model.add(Conv2D(filters = 32, kernel_size = (3, 3), padding='same'))
    model.add(LeakyReLU(0.1))
    
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 32, kernel_size = (3, 3), padding='same'))
    model.add(LeakyReLU(0.1))
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding='same'))
    model.add(LeakyReLU(0.1))

    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    
    model.add(Dense(256))
    model.add(LeakyReLU(0.1))

    model.add(Dropout(0.5))
    
    model.add(Dense(10))
    model.add(Activation("softmax"))
    
    return model

In [ ]:
# Summarize a model
model = make_model()
model.summary()

# **Define the loss function, the optimizer and the metrics**

In [ ]:
INIT_LR = 5e-3 # Learning rate, is default schedule in all Keras Optimizers
BATCH_SIZE = 32 # Batch size is a hyperparameter of gradient descent that controls the number of training samples
EPOCHS = 80 # Number of epochs

# Create object of deep learning model
model = make_model()

# Define the loss function, the optimizer and the metrics
model.compile(
    loss='categorical_crossentropy',  
    optimizer=keras.optimizers.Adamax(lr=INIT_LR), 
    metrics=['accuracy']  # report accuracy during training
)

# Scheduler of learning rate (decay with epochs)
def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch

# Callback for printing of actual learning rate used by optimizer
class LrHistory(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        print("Learning rate:", K.get_value(model.optimizer.lr))

# **Fit the Deep Learning Mode**

In [ ]:
import tensorflow_addons as tfa
tqdm_callback = tfa.callbacks.TQDMProgressBar()

model.fit(
    x_train2, y_train2,  # Prepared data
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler), 
               LrHistory(), 
               tqdm_callback],
    shuffle=True,
    verbose=0,
    initial_epoch=0
)